In [43]:
import pandas as pd
from datetime import datetime, timedelta
import calendar

start_year = int(input("Enter start year: "))
end_year = int(input("Enter end year: "))

def is_leap_year_persian(year):
    return (year % 33) in [1, 5, 9, 13, 17, 21, 25, 29]

def jalali_to_gregorian(jy, jm, jd):
    jy += 1595
    days = -355668 + (365*jy) + ((jy//33)*8) + (((jy%33)+3)//4) + (31*(jm-1) if jm < 7 else (186 + (jm-7)*30)) + jd
    gy = 400 * (days // 146097)
    days %= 146097
    leap = True
    if days > 36524:
        gy += 100*(days//36524)
        days %= 36524
        if days >= 365:
            days += 1
        else:
            leap = False
    gy += 4*(days//1461)
    days %= 1461
    if days > 365:
        gy += (days-1)//365
        days = (days-1)%365
        leap = False
    gd_m = [0,31,59,90,120,151,181,212,243,273,304,334]
    for i in range(1,13):
        v = gd_m[i-1] + (1 if i > 2 and leap else 0)
        if i == 12 or days < gd_m[i] + (1 if i+1 > 2 and leap else 0):
            return gy, i, days - v + 1

def day_of_week(gy, gm, gd):
    if gm < 3:
        gm += 12
        gy -= 1
    K, J = gy % 100, gy // 100
    h = (gd + (13*(gm + 1))//5 + K + (K//4) + (J//4) + (5*J)) % 7
    return (["شنبه","یکشنبه","دوشنبه","سه‌شنبه","چهارشنبه","پنج‌شنبه","جمعه"][h],
            ["Saturday","Sunday","Monday","Tuesday","Wednesday","Thursday","Friday"][h], h + 1)

persian_months = {1:"فروردین",2:"اردیبهشت",3:"خرداد",4:"تیر",5:"مرداد",6:"شهریور",7:"مهر",8:"آبان",9:"آذر",10:"دی",11:"بهمن",12:"اسفند"}
gregorian_months = {1:"January",2:"February",3:"March",4:"April",5:"May",6:"June",7:"July",8:"August",9:"September",10:"October",11:"November",12:"December"}
seasons = {1:"بهار",2:"بهار",3:"بهار",4:"تابستان",5:"تابستان",6:"تابستان",7:"پاییز",8:"پاییز",9:"پاییز",10:"زمستان",11:"زمستان",12:"زمستان"}
half_years = {1:"نیمسال اول",2:"نیمسال اول",3:"نیمسال اول",4:"نیمسال اول",5:"نیمسال اول",6:"نیمسال اول",7:"نیمسال دوم",8:"نیمسال دوم",9:"نیمسال دوم",10:"نیمسال دوم",11:"نیمسال دوم",12:"نیمسال دوم"}
gregorian_events_en = {(1,1):"New Year's Day",(1,4):"World Braille Day",(1,24):"International Day of Education",(2,2):"World Wetlands Day",(2,13):"World Radio Day",(2,21):"International Mother Language Day",(3,1):"Zero Discrimination Day",(3,8):"International Women's Day",(3,21):"International Day of Nowruz",(3,22):"World Water Day",(4,7):"World Health Day",(4,22):"International Mother Earth Day",(5,15):"International Day of Families",(6,5):"World Environment Day",(6,20):"World Refugee Day",(7,11):"World Population Day",(8,12):"International Youth Day",(9,21):"International Day of Peace",(10,1):"International Day of Older Persons",(10,5):"World Teachers' Day",(11,20):"World Children's Day",(12,10):"Human Rights Day",(12,25):"Christmas Day"}
gregorian_events_fa = {(1,1):"روز سال نو",(1,4):"روز جهانی بریل",(1,24):"روز جهانی آموزش",(2,2):"روز جهانی تالاب‌ها",(2,13):"روز جهانی رادیو",(2,21):"روز جهانی زبان مادری",(3,1):"روز جهانی بدون تبعیض",(3,8):"روز جهانی زنان",(3,21):"روز جهانی نوروز",(3,22):"روز جهانی آب",(4,7):"روز جهانی سلامت",(4,22):"روز جهانی زمین مادر",(5,15):"روز جهانی خانواده",(6,5):"روز جهانی محیط زیست",(6,20):"روز جهانی پناهندگان",(7,11):"روز جهانی جمعیت",(8,12):"روز جهانی جوانان",(9,21):"روز جهانی صلح",(10,1):"روز جهانی سالمندان",(10,5):"روز جهانی معلمان",(11,20):"روز جهانی کودک",(12,10):"روز جهانی حقوق بشر",(12,25):"روز کریسمس"}
persian_events = {(1,1):"عید نوروز",(1,2):"عید نوروز",(1,3):"عید نوروز",(1,4):"عید نوروز",(1,12):"روز جمهوری اسلامی",(1,13):"روز طبیعت",(3,14):"رحلت امام",(3,15):"قیام 15 خرداد",(11,22):"پیروزی انقلاب",(12,29):"روز ملی شدن نفت"}
persian_holidays = {(1,1):1,(1,2):1,(1,3):1,(1,4):1,(1,13):1,(3,14):1,(3,15):1,(11,22):1,(12,29):1}
gregorian_holidays = {(1,1):1,(12,25):1}

data = []
for year in range(start_year, end_year + 1):
    for month in range(1, 13):
        days_in_month = 31 if month <= 6 else (30 if month <= 11 else (30 if is_leap_year_persian(year) else 29))
        for day in range(1, days_in_month + 1):
            gy, gm, gd = jalali_to_gregorian(year, month, day)
            persian_day, english_day, day_id = day_of_week(gy, gm, gd)
            greg_date = datetime(gy, gm, gd)
            get_season = lambda m: "Winter" if m in [12,1,2] else ("Spring" if m in [3,4,5] else ("Summer" if m in [6,7,8] else "Autumn"))
            data.append({
                'shamsi_date': int(f"{year:04d}{month:02d}{day:02d}"),
                'miladi_date': f"{gy:04d}-{gm:02d}-{gd:02d}",
                'miladi_day_of_week_title': english_day,
                'shamsi_date_title': f"{year:04d}/{month:02d}/{day:02d}",
                'shamsi_next_date': int(f"{year:04d}{month:02d}{day+1:02d}") if day < days_in_month else (int(f"{year:04d}{month+1:02d}01") if month < 12 else int(f"{year+1:04d}0101")),
                'shamsi_month_id': int(f"{year:04d}{month:02d}"),
                'shamsi_month_title': f"{persian_months[month]} {year}",
                'shamsi_month_of_year_title': persian_months[month],
                'shamsi_season_id': int(f"{year:04d}{month:02d}"),
                'shamsi_season_title': f"{seasons[month]} {year}",
                'shamsi_half_year_id': int(f"{year:04d}{month:02d}"),
                'shamsi_half_year_title': f"{half_years[month]} {year}",
                'shamsi_year_id': year,
                'shamsi_month_of_year_id': month,
                'shamsi_day_of_month_id': day,
                'shamsi_day_of_year_id': sum([31 if m<=6 else (30 if m<=11 else (30 if is_leap_year_persian(year) else 29)) for m in range(1,month)]) + day,
                'shamsi_day_of_week_title': persian_day,
                'shamsi_day_of_week_id': day_id,
                'shamsi_event_name': persian_events.get((month, day)),
                'shamsi_is_holiday': persian_holidays.get((month, day), 0),
                'shamsi_is_happy_holiday': persian_holidays.get((month, day), 0),
                'shamsi_is_sad_holiday': 0,
                'shamsi_is_weekend': 1 if day_id in [6, 7] else 0,
                'gregorian_next_date': (greg_date + timedelta(days=1)).strftime('%Y-%m-%d'),
                'gregorian_month_id': int(f"{gy:04d}{gm:02d}"),
                'gregorian_month_title': f"{gregorian_months[gm]} {gy}",
                'gregorian_month_of_year_title': gregorian_months[gm],
                'gregorian_season_id': int(f"{gy:04d}{gm:02d}"),
                'gregorian_season_title': f"{get_season(gm)} {gy}",
                'gregorian_half_year_id': int(f"{gy:04d}{gm:02d}"),
                'gregorian_half_year_title': f"{'H1' if gm <= 6 else 'H2'} {gy}",
                'gregorian_year_id': gy,
                'gregorian_month_of_year_id': gm,
                'gregorian_day_of_month_id': gd,
                'gregorian_day_of_year_id': greg_date.timetuple().tm_yday,
                'gregorian_day_of_week_title': english_day,
                'gregorian_day_of_week_id': (greg_date.weekday() + 2) % 7 + 1,
                'gregorian_event_name_en': gregorian_events_en.get((gm, gd)),
                'gregorian_event_name_fa': gregorian_events_fa.get((gm, gd)),
                'gregorian_is_holiday': gregorian_holidays.get((gm, gd), 0),
                'gregorian_is_happy_holiday': gregorian_holidays.get((gm, gd), 0),
                'gregorian_is_sad_holiday': 0,
                'gregorian_is_weekend': 1 if greg_date.weekday() in [5, 6] else 0,
            })

df = pd.DataFrame(data)

def assign_monthly_weeks(df_month):
    df_month = df_month.sort_values('shamsi_date').copy()
    is_saturday = (df_month['shamsi_day_of_week_id'] == 1)
    week_break = is_saturday.astype(int)
    if len(week_break) > 0:
        week_break.iloc[0] = 0
    df_month['week_num_in_month'] = 1 + week_break.cumsum()
    df_month['shamsi_week_id'] = df_month['shamsi_month_id'].astype(str) + '_' + df_month['week_num_in_month'].astype(int).astype(str)
    return df_month

def assign_yearly_weeks(df_year):
    df_year = df_year.sort_values(['shamsi_month_id','shamsi_date']).copy()
    is_saturday = (df_year['shamsi_day_of_week_id'] == 1)
    week_break = is_saturday.astype(int)
    if len(week_break) > 0:
        week_break.iloc[0] = 0
    df_year['week_num_in_year'] = 1 + week_break.cumsum()
    return df_year

df = df.sort_values(['shamsi_month_id','shamsi_date']).groupby('shamsi_month_id', group_keys=False).apply(assign_monthly_weeks)
df = df.groupby('shamsi_year_id', group_keys=False).apply(assign_yearly_weeks)
df['shamsi_week_title'] = df.apply(lambda row: f"هفته {int(row['week_num_in_year'])} سال {int(row['shamsi_year_id'])}", axis=1)
df['shamsi_week_of_year_id'] = df.apply(lambda row: f"{int(row['shamsi_year_id']):04d}_{int(row['week_num_in_year']):02d}", axis=1)

dates = pd.to_datetime(df['miladi_date'])
iso = dates.dt.isocalendar()
df['gregorian_week_id'] = iso['week'].astype(int)
df['gregorian_week_title'] = df.apply(lambda row: f"Week {row['gregorian_week_id']} {row['gregorian_year_id']}", axis=1)
df['gregorian_week_of_year_id'] = df.apply(lambda row: f"{int(row['gregorian_year_id']):04d}_{int(row['gregorian_week_id']):02d}", axis=1)
df['miladi_week_id'] = iso['week'].astype(int)
df['miladi_week_num_in_month'] = df.groupby(dates.dt.to_period('M').astype(str))['miladi_week_id'].transform(lambda s: s.rank(method='dense').astype(int))

final_columns = ['shamsi_date','miladi_date','miladi_day_of_week_title','shamsi_date_title','shamsi_next_date','shamsi_month_id','shamsi_month_title','shamsi_month_of_year_title','shamsi_season_id','shamsi_season_title','shamsi_half_year_id','shamsi_half_year_title','shamsi_year_id','shamsi_month_of_year_id','shamsi_day_of_month_id','shamsi_day_of_year_id','shamsi_day_of_week_title','shamsi_week_id','shamsi_week_title','shamsi_week_of_year_id','shamsi_day_of_week_id','shamsi_event_name','shamsi_is_holiday','shamsi_is_happy_holiday','shamsi_is_sad_holiday','shamsi_is_weekend','gregorian_next_date','gregorian_month_id','gregorian_month_title','gregorian_month_of_year_title','gregorian_season_id','gregorian_season_title','gregorian_half_year_id','gregorian_half_year_title','gregorian_year_id','gregorian_month_of_year_id','gregorian_day_of_month_id','gregorian_day_of_year_id','gregorian_day_of_week_title','gregorian_week_id','gregorian_week_title','gregorian_week_of_year_id','gregorian_day_of_week_id','gregorian_event_name_en','gregorian_event_name_fa','gregorian_is_holiday','gregorian_is_happy_holiday','gregorian_is_sad_holiday','gregorian_is_weekend','miladi_week_id','miladi_week_num_in_month']
df_final = df[final_columns].copy()

print(f"Total days: {len(df_final):,}")
print(df_final.head())

filename = f"date_dimension_{df_final['shamsi_year_id'].min()}_{df_final['shamsi_year_id'].max()}.xlsx"
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    df_final.to_excel(writer, sheet_name='Date_Dimension', index=False)
    worksheet = writer.sheets['Date_Dimension']
    for column in worksheet.columns:
        max_length = max([len(str(cell.value)) for cell in column if cell.value] + [0])
        worksheet.column_dimensions[column[0].column_letter].width = min(max_length + 2, 50)

print(f"✅ Saved: {filename}")


C:\Users\pc\AppData\Local\Temp\ipykernel_85668\477800570.py:131: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.sort_values(['shamsi_month_id','shamsi_date']).groupby('shamsi_month_id', group_keys=False).apply(assign_monthly_weeks)
C:\Users\pc\AppData\Local\Temp\ipykernel_85668\477800570.py:132: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('shamsi_year_id', group_keys=False).apply(assign_yearl

Total days: 730
   shamsi_date miladi_date miladi_day_of_week_title shamsi_date_title  \
0     14000101  2021-03-21                   Sunday        1400/01/01   
1     14000102  2021-03-22                   Monday        1400/01/02   
2     14000103  2021-03-23                  Tuesday        1400/01/03   
3     14000104  2021-03-24                Wednesday        1400/01/04   
4     14000105  2021-03-25                 Thursday        1400/01/05   

   shamsi_next_date  shamsi_month_id shamsi_month_title  \
0          14000102           140001       فروردین 1400   
1          14000103           140001       فروردین 1400   
2          14000104           140001       فروردین 1400   
3          14000105           140001       فروردین 1400   
4          14000106           140001       فروردین 1400   

  shamsi_month_of_year_title  shamsi_season_id shamsi_season_title  \
0                    فروردین            140001           بهار 1400   
1                    فروردین            140001    

In [49]:
df_final.tail(22)

,shamsi_date,miladi_date,miladi_day_of_week_title,shamsi_date_title,shamsi_next_date,shamsi_month_id,shamsi_month_title,shamsi_month_of_year_title,shamsi_season_id,shamsi_season_title,shamsi_half_year_id,shamsi_half_year_title,shamsi_year_id,shamsi_month_of_year_id,shamsi_day_of_month_id,shamsi_day_of_year_id,shamsi_day_of_week_title,shamsi_week_id,shamsi_week_title,shamsi_week_of_year_id,shamsi_day_of_week_id,shamsi_event_name,shamsi_is_holiday,shamsi_is_happy_holiday,shamsi_is_sad_holiday,shamsi_is_weekend,gregorian_next_date,gregorian_month_id,gregorian_month_title,gregorian_month_of_year_title,gregorian_season_id,gregorian_season_title,gregorian_half_year_id,gregorian_half_year_title,gregorian_year_id,gregorian_month_of_year_id,gregorian_day_of_month_id,gregorian_day_of_year_id,gregorian_day_of_week_title,gregorian_week_id,gregorian_week_title,gregorian_week_of_year_id,gregorian_day_of_week_id,gregorian_event_name_en,gregorian_event_name_fa,gregorian_is_holiday,gregorian_is_happy_holiday,gregorian_is_sad_holiday,gregorian_is_weekend,miladi_week_id,miladi_week_num_in_month
708,14011208,2023-02-27,Monday,1401/12/08,14011209,140112,اسفند 1401,اسفند,140112,زمستان 1401,140112,نیمسال دوم 1401,1401,12,8,344,دوشنبه,140112_2,هفته 50 سال 1401,1401_50,3,None,0,0,0,0,2023-02-28,202302,February 2023,February,202302,Winter 2023,202302,H1 2023,2023,2,27,58,Monday,9,Week 9 2023,2023_09,3,None,None,0,0,0,0,9,5
709,14011209,2023-02-28,Tuesday,1401/12/09,14011210,140112,اسفند 1401,اسفند,140112,زمستان 1401,140112,نیمسال دوم 1401,1401,12,9,345,سه‌شنبه,140112_2,هفته 50 سال 1401,1401_50,4,None,0,0,0,0,2023-03-01,202302,February 2023,February,202302,Winter 2023,202302,H1 2023,2023,2,28,59,Tuesday,9,Week 9 2023,2023_09,4,None,None,0,0,0,0,9,5
710,14011210,2023-03-01,Wednesday,1401/12/10,14011211,140112,اسفند 1401,اسفند,140112,زمستان 1401,140112,نیمسال دوم 1401,1401,12,10,346,چهارشنبه,140112_2,هفته 50 سال 1401,1401_50,5,None,0,0,0,0,2023-03-02,202303,March 2023,March,202303,Spring 2023,202303,H1 2023,2023,3,1,60,Wednesday,9,Week 9 2023,2023_09,5,Zero Discrimination Day,روز جهانی بدون تبعیض,0,0,0,0,9,1
711,14011211,2023-03-02,Thursday,1401/12/11,14011212,140112,اسفند 1401,اسفند,140112,زمستان 1401,140112,نیمسال دوم 1401,1401,12,11,347,پنج‌شنبه,140112_2,هفته 50 سال 1401,1401_50,6,None,0,0,0,1,2023-03-03,202303,March 2023,March,202303,Spring 2023,202303,H1 2023,2023,3,2,61,Thursday,9,Week 9 2023,2023_09,6,None,None,0,0,0,0,9,1
712,14011212,2023-03-03,Friday,1401/12/12,14011213,140112,اسفند 1401,اسفند,140112,زمستان 1401,140112,نیمسال دوم 1401,1401,12,12,348,جمعه,140112_2,هفته 50 سال 1401,1401_50,7,None,0,0,0,1,2023-03-04,202303,March 2023,March,202303,Spring 2023,202303,H1 2023,2023,3,3,62,Friday,9,Week 9 2023,2023_09,7,None,None,0,0,0,0,9,1
713,14011213,2023-03-04,Saturday,1401/12/13,14011214,140112,اسفند 1401,اسفند,140112,زمستان 1401,140112,نیمسال دوم 1401,1401,12,13,349,شنبه,140112_3,هفته 51 سال 1401,1401_51,1,None,0,0,0,0,2023-03-05,202303,March 2023,March,202303,Spring 2023,202303,H1 2023,2023,3,4,63,Saturday,9,Week 9 2023,2023_09,1,None,None,0,0,0,1,9,1
714,14011214,2023-03-05,Sunday,1401/12/14,14011215,140112,اسفند 1401,اسفند,140112,زمستان 1401,140112,نیمسال دوم 1401,1401,12,14,350,یکشنبه,140112_3,هفته 51 سال 1401,1401_51,2,None,0,0,0,0,2023-03-06,202303,March 2023,March,202303,Spring 2023,202303,H1 2023,2023,3,5,64,Sunday,9,Week 9 2023,2023_09,2,None,None,0,0,0,1,9,1
715,14011215,2023-03-06,Monday,1401/12/15,14011216,140112,اسفند 1401,اسفند,140112,زمستان 1401,140112,نیمسال دوم 1401,1401,12,15,351,دوشنبه,140112_3,هفته 51 سال 1401,1401_51,3,None,0,0,0,0,2023-03-07,202303,March 2023,March,202303,Spring 2023,202303,H1 2023,2023,3,6,65,Monday,10,Week 10 2023,2023_10,3,None,None,0,0,0,0,10,2
716,14011216,2023-03-07,Tuesday,1401/12/16,14011217,140112,اسفند 1401,اسفند,140112,زمستان 1401,140112,نیمسال دوم 1401,1401,12,16,352,سه‌شنبه,140112_3,هفته 51 سال 1401,1401_51,4,None,0,0,0,0,2023-03-08,202303,March 2023,March,202303,Spring 2023,202303,H1 2023,2023,